In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 9 class
predictions = Dense(9, activation='softmax')(x)

In [4]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [7]:
# model.summary()

### prepare data for model input

In [8]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [9]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [10]:
train_generator = train_datagen.flow_from_directory(
        'F:/abuzar/data/raw_data',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size)

Found 65 images belonging to 9 classes.


In [11]:
model.fit_generator(train_generator, epochs=15, steps_per_epoch=10)

Epoch 1/15
10/10 [==============================] - 28s - loss: 6.8701    
Epoch 2/15
10/10 [==============================] - 24s - loss: 4.2397    
Epoch 3/15
10/10 [==============================] - 26s - loss: 3.3945    
Epoch 4/15
10/10 [==============================] - 24s - loss: 3.3593    
Epoch 5/15
10/10 [==============================] - 26s - loss: 2.5462    
Epoch 6/15
10/10 [==============================] - 23s - loss: 2.6196    
Epoch 7/15
10/10 [==============================] - 26s - loss: 3.2983    
Epoch 8/15
10/10 [==============================] - 26s - loss: 2.5578    
Epoch 9/15
10/10 [==============================] - 24s - loss: 2.7596    
Epoch 10/15
10/10 [==============================] - 26s - loss: 2.5077    
Epoch 11/15
10/10 [==============================] - 25s - loss: 2.3674    
Epoch 12/15
10/10 [==============================] - 25s - loss: 2.4355    
Epoch 13/15
10/10 [==============================] - 25s - loss: 3.0271    
Epoch 14/15
10/10 [==

## Get prediction with new model

In [13]:
from keras.applications.inception_v3 import preprocess_input, decode_predictions

In [14]:
# Image Preprocessing
img_path = 'F:/cifar10/abc.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [15]:
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 9)